## Version Description
In this version, we deploy a Hybrid PINN with adaptive Line Search in _single data point training_ mode, to predict 8 correction factors in TZ3, viz. for the MTR, the 4 riser ducts and other associated elements

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# import required libraries
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource,HoverTool
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.io import export_svgs
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [3]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [4]:
import re 
def sorted_alphanum(l): 
    """ Sort the given alphanumeric list""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

### Data Acquisition

In [5]:
target_df = pd.read_csv('../data/TZ3_dataset.csv')
target_df.head()

,HoV,MIXP,MIXT,AMBP,AMBT,Density(kg/m³),CAOLH_SumFlow,CAORH_SumFlow,LAOLH_SumFlow,LAORH_SumFlow,...,LAO_335_FP2,R300_HD,R310_HS1,R311_HS1,R312_HS1,R313_HS1,R320_HS1,R320_HS2,R321_HS1,R321_HS2
0,A1,2600.0000,293.150000,101401.6000,299.386667,1.2207,144.874607,146.222707,73.311838,71.571423,...,1,116,60,48,128,125,113,38,113,40
1,A2,2600.0000,293.150000,101576.3000,298.448667,1.2228,153.137249,144.985186,61.077974,65.831902,...,1,114,60,40,140,138,105,35,113,39
2,A3,2606.1928,298.101330,102136.6035,297.109024,1.2090,156.559508,160.021532,81.965312,87.345475,...,1,124,63,51,122,120,113,39,113,40
3,A4,2599.8998,294.390392,103195.6642,295.060027,1.2368,153.335489,148.015738,89.216361,80.024263,...,1,124,55,59,120,120,113,43,113,43
4,A5,2600.0000,293.150000,102856.2000,294.755833,1.2380,146.197707,148.581402,87.030343,86.772646,...,1,122,75,60,125,120,113,38,113,40


In [6]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 32 columns):
HoV               30 non-null object
MIXP              30 non-null float64
MIXT              30 non-null float64
AMBP              30 non-null float64
AMBT              30 non-null float64
Density(kg/m³)    30 non-null float64
CAOLH_SumFlow     30 non-null float64
CAORH_SumFlow     30 non-null float64
LAOLH_SumFlow     30 non-null float64
LAORH_SumFlow     30 non-null float64
TZ3_Flow          30 non-null float64
LAO_330_FM        30 non-null int64
LAO_332_FM        30 non-null int64
LAO_334_FM        30 non-null int64
LAO_331_FM        30 non-null int64
LAO_333_FM        30 non-null int64
LAO_335_FM        30 non-null int64
LAO_330_FP2       30 non-null int64
LAO_332_FP2       30 non-null int64
LAO_334_FP2       30 non-null int64
LAO_331_FP2       30 non-null int64
LAO_333_FP2       30 non-null int64
LAO_335_FP2       30 non-null int64
R300_HD           30 non-null int64
R310_HS1    

In [7]:
# Define Restrictors and their parameters
MTR = 'R300_HD'
CLRs =  ['R310_HS1', 'R311_HS1', 'R312_HS1', 'R313_HS1']
CAORs = ['R320_HS1', 'R320_HS2', 'R321_HS1', 'R321_HS2']
MHRS = CLRs + CAORs

# Duct Areas
MHR_Duct_Areas = {'R310_HS1':0.0129658,'R311_HS1':0.01327322896,'R312_HS1':0.01767145868,'R313_HS1':0.01767145868,
                  'R320_HS1':0.0108506,'R320_HS2':0.00470159,'R321_HS1':0.0108506,'R321_HS2':0.00470158}

restrictor_duct_areas = {'R300_HD':0.02405281875, 'riser_LAO_RHS':0.00516612, 'riser_LAO_LHS':0.0052845, 'riser_CAO_RHS':0.0173304, 'riser_CAO_LHS':0.0173301 }

#3D Correction Factors for MultiHole Restrictors
MTR_cf = 0.812
MHR_cf = {'R310_HS1':0.832, 'R311_HS1':0.879, 'R312_HS1':0.727, 'R313_HS1':0.771, 
          'R320_HS1':0.912, 'R320_HS2':0.974, 'R321_HS1':0.817, 'R321_HS2':1.047}


## Define Initial Zeta values for riser ducts from CFD

riser_duct_zetasCFD = { 'riser_LAO_RHS':0.730110153034089, 'riser_LAO_LHS':1.91790022798789, 'riser_CAO_RHS':11.4767551741128, 'riser_CAO_LHS':11.8379787151916}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [8]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [9]:
_,target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,restrictor_duct_areas['R300_HD'],MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R300_HD_Zeta3D']].head()

,R300_HD,R300_HD_Zeta3D
0,116,5.076081
1,114,5.642979
2,124,3.307892
3,124,3.307892
4,122,3.685326


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [10]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [11]:
# Calculate Zeta for Multi-Hole Restrictors
hole_dia = 8 # MHR Hole Dia
for clr in CLRs:
    MHR_nr_holes = target_df[clr].values
    _, target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele, hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORs:
    CAOR_nr_holes = target_df[caor].values
    _,target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

**Store Loss Coefficient Values and Names for all Restrictors**:

In [12]:
# Gather all column names required for the input loss coefficients
from TZ3_FDDN_Lib import zeta_values, name_variables

In [13]:
print('FDDN Elements:\n',name_variables)

FDDN Elements:
 ['TZ-3_zmix', 'MTR-300', 'riser_LAO_RHS', 'riser_LAO_LHS', 'riser_CAO_RHS', 'riser_CAO_LHS', 'R-310', 'R-311', 'R-312', 'R-313', 'LAOR-330', 'LAOR-332', 'LAOR-334', 'LAOR-336', 'LAOR-331', 'LAOR-333', 'LAOR-335', 'LAOR-337', 'CAO-320_fwd', 'CAO-320_mid', 'CAO-320_aft', 'CAO-321_fwd', 'CAO-321_mid', 'CAO-321_aft', 'LAO-330', 'LAO-332', 'LAO-334', 'LAO-336', 'LAO-331', 'LAO-333', 'LAO-335', 'LAO-337', 'CAOR-320_fwd', 'CAOR-320_mid', 'CAOR-320_aft', 'CAOR-321_fwd', 'CAOR-321_mid', 'CAOR-321_aft', 'pp2310pdloss', 'pp2311pdloss', 'pp2312pdloss', 'pp2313pdloss', '23120-320_F', '23120-320_M', '23120-320_A', '23130-321_F', '23130-321_M', '23130-321_A', '3302.2-3303', '3303-3304', '3302.2-3305', '3305-3306', '2301-3302.1', '2301-3302.2']


In [14]:
cf_riser_initial = 1
idx = target_df.columns.get_loc("R300_HD_Zeta3D")
target_df.insert(loc=(idx+1), column='riser_LAO_RHS_Zeta3D', value= float(zeta_values[2]) * cf_riser_initial)
target_df.insert(loc=(idx+2), column='riser_LAO_LHS_Zeta3D', value= float(zeta_values[3]) * cf_riser_initial)
target_df.insert(loc=(idx+3), column='riser_CAO_RHS_Zeta3D', value= float(zeta_values[4]) * cf_riser_initial)
target_df.insert(loc=(idx+4), column='riser_CAO_LHS_Zeta3D', value= float(zeta_values[5]) * cf_riser_initial)
target_df['riser_3302_2_3305_Zeta3D']= float(zeta_values[50]) * cf_riser_initial
target_df['riser_3302_1_Zeta3D']= float(zeta_values[52]) * cf_riser_initial
target_df['riser_3302_2_Zeta3D']= float(zeta_values[53]) * cf_riser_initial

In [15]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
fm_flag_col_names = sorted_alphanum([col for col in target_df.columns if 'FM' in col])
fm_flag_col_names_idx = [target_df.columns.get_loc(col) for col in fm_flag_col_names if col in target_df]
fp2_flag_col_names = sorted_alphanum([col for col in target_df.columns if 'FP2' in col])
fp2_flag_col_names_idx = [target_df.columns.get_loc(col) for col in fp2_flag_col_names if col in target_df]
fddn_input_col_names = fm_flag_col_names + fp2_flag_col_names + zeta_col_names
print(fddn_input_col_names)

['LAO_330_FM', 'LAO_331_FM', 'LAO_332_FM', 'LAO_333_FM', 'LAO_334_FM', 'LAO_335_FM', 'LAO_330_FP2', 'LAO_331_FP2', 'LAO_332_FP2', 'LAO_333_FP2', 'LAO_334_FP2', 'LAO_335_FP2', 'R300_HD_Zeta3D', 'riser_LAO_RHS_Zeta3D', 'riser_LAO_LHS_Zeta3D', 'riser_CAO_RHS_Zeta3D', 'riser_CAO_LHS_Zeta3D', 'R310_HS1_Zeta3D', 'R311_HS1_Zeta3D', 'R312_HS1_Zeta3D', 'R313_HS1_Zeta3D', 'R320_HS1_Zeta3D', 'R320_HS2_Zeta3D', 'R321_HS1_Zeta3D', 'R321_HS2_Zeta3D', 'riser_3302_2_3305_Zeta3D', 'riser_3302_1_Zeta3D', 'riser_3302_2_Zeta3D']


In [16]:
target_df[zeta_col_names]

,R300_HD_Zeta3D,riser_LAO_RHS_Zeta3D,riser_LAO_LHS_Zeta3D,riser_CAO_RHS_Zeta3D,riser_CAO_LHS_Zeta3D,R310_HS1_Zeta3D,R311_HS1_Zeta3D,R312_HS1_Zeta3D,R313_HS1_Zeta3D,R320_HS1_Zeta3D,R320_HS2_Zeta3D,R321_HS1_Zeta3D,R321_HS2_Zeta3D,riser_3302_2_3305_Zeta3D,riser_3302_1_Zeta3D,riser_3302_2_Zeta3D
0,5.076081,0.73011,1.9179,11.476755,11.837979,28.753818,54.552445,7.864128,8.909238,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
1,5.642979,0.73011,1.9179,11.476755,11.837979,28.753818,82.805829,6.089576,6.733670,4.124839,9.755073,2.885978,7.658574,1.880442,1.291464,2.508788
2,3.307892,0.73011,1.9179,11.476755,11.837979,25.516955,47.357346,8.982575,9.966643,3.221556,7.124641,2.885978,7.097864,1.880442,1.291464,2.508788
3,3.307892,0.73011,1.9179,11.476755,11.837979,35.466354,33.489790,9.397860,9.966643,3.221556,5.281530,2.885978,5.677335,1.880442,1.291464,2.508788
4,3.685326,0.73011,1.9179,11.476755,11.837979,16.449171,32.156533,8.400799,9.966643,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
5,3.685326,0.73011,1.9179,11.476755,11.837979,16.449171,39.622740,10.544011,11.999959,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
6,4.102714,0.73011,1.9179,11.476755,11.837979,28.753818,54.552445,7.864128,8.909238,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
7,5.076081,0.73011,1.9179,11.476755,11.837979,35.466354,63.319432,4.402668,5.263191,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
8,5.076081,0.73011,1.9179,11.476755,11.837979,35.466354,63.319432,4.402668,5.263191,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
9,4.102714,0.73011,1.9179,11.476755,11.837979,22.740948,63.319432,6.217884,6.876629,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788


### Training data Preparation

In [17]:
from TZ3_8CF import NormbyMax, PINN

In [18]:
features2scale = ['R300_HD_Zeta3D','riser_3302_1_Zeta3D','riser_3302_2_Zeta3D','riser_3302_2_3305_Zeta3D','riser_LAO_RHS_Zeta3D','riser_LAO_LHS_Zeta3D','riser_CAO_RHS_Zeta3D','riser_CAO_LHS_Zeta3D','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow','TZ3_Flow']
features_max, df_rescaled = NormbyMax(target_df,features2scale)
print('Max value of all features:',features_max)

Max value of all features: {'R300_HD_Zeta3D': 5.642978728418095, 'riser_3302_1_Zeta3D': 1.29146402905416, 'riser_3302_2_Zeta3D': 2.50878846999567, 'riser_3302_2_3305_Zeta3D': 1.88044178924511, 'riser_LAO_RHS_Zeta3D': 0.730110153034089, 'riser_LAO_LHS_Zeta3D': 1.91790022798789, 'riser_CAO_RHS_Zeta3D': 11.4767551741128, 'riser_CAO_LHS_Zeta3D': 11.8379787151916, 'LAORH_SumFlow': 104.206563, 'LAOLH_SumFlow': 99.792513, 'CAORH_SumFlow': 160.021532, 'CAOLH_SumFlow': 159.438776, 'TZ3_Flow': 505.45200199999994}


In [19]:
features_max_df = pd.DataFrame(data = features_max, index=[0])
print('Max value of all input features for normalization')
features_max_df

Max value of all input features for normalization


,R300_HD_Zeta3D,riser_3302_1_Zeta3D,riser_3302_2_Zeta3D,riser_3302_2_3305_Zeta3D,riser_LAO_RHS_Zeta3D,riser_LAO_LHS_Zeta3D,riser_CAO_RHS_Zeta3D,riser_CAO_LHS_Zeta3D,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow,TZ3_Flow
0,5.642979,1.291464,2.508788,1.880442,0.73011,1.9179,11.476755,11.837979,104.206563,99.792513,160.021532,159.438776,505.452002


In [20]:
V_max_org = features_max_df[['TZ3_Flow','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow']]
V_max_org

,TZ3_Flow,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow
0,505.452002,104.206563,99.792513,160.021532,159.438776


In [21]:
# Store indices and values of train data
input_features = ['R300_HD_Zeta3D']
train_data = df_rescaled[input_features].iloc[[12]] #.iloc[[1, 3, 6, 9, 11, 13, 15, 18, 21, 24, 28, 30]]
train_data_idx = train_data.index.values.tolist()
train_R300HD_series   = target_df['R300_HD'].iloc[train_data_idx]
train_riser_LAO_RHS_Zeta3D_series = target_df['riser_LAO_RHS_Zeta3D'].iloc[train_data_idx]
train_riser_LAO_LHS_Zeta3D_series = target_df['riser_LAO_LHS_Zeta3D'].iloc[train_data_idx]
train_riser_CAO_RHS_Zeta3D_series = target_df['riser_CAO_RHS_Zeta3D'].iloc[train_data_idx]
train_riser_CAO_LHS_Zeta3D_series = target_df['riser_CAO_LHS_Zeta3D'].iloc[train_data_idx]
train_riser_3302_2_3305_Zeta3D_series = target_df['riser_3302_2_3305_Zeta3D'].iloc[train_data_idx]
train_riser_3302_1_Zeta3D_series = target_df['riser_3302_1_Zeta3D'].iloc[train_data_idx]
train_riser_3302_2_Zeta3D_series = target_df['riser_3302_2_Zeta3D'].iloc[train_data_idx]

print("Number of training samples:", len(train_data),';\tTrain data Shape:',train_data.shape,'\nSCALED FEATURES:')
train_data

Number of training samples: 1 ;	Train data Shape: (1, 1) 
SCALED FEATURES:


,R300_HD_Zeta3D
12,0.653082


### Predict $c_f$ with Physics Informed Neural Network

In [22]:
### Set the hyperparameters here ###
learning_rates = [0, 0.001] ### IMPORTANT PARAMETER [0, 0.0001]
N_i = train_data.shape[1] # No. of feature columns in the dataframe
hl_dim = [9] ### IMPORTANT PARAMETER
output_nodes = 8
network_dim = (N_i,hl_dim,output_nodes)

epsi_factor = 0.1 # 0.01
MTR_epsi = np.float(MTR_cf * epsi_factor)
riser_LAO_RHS_epsi = np.float(1 * epsi_factor)
riser_LAO_LHS_epsi = np.float(1 * epsi_factor)
riser_CAO_RHS_epsi = np.float(1 * epsi_factor)
riser_CAO_LHS_epsi = np.float(1 * epsi_factor)
riser_3302_2_3305_epsi = np.float(1 * epsi_factor)
riser_3302_1_epsi = np.float(1 * epsi_factor)
riser_3302_2_epsi = np.float(1 * epsi_factor)
epsi_tuple = (MTR_epsi, riser_LAO_RHS_epsi, riser_LAO_LHS_epsi, riser_CAO_RHS_epsi, riser_CAO_LHS_epsi, riser_3302_2_3305_epsi, riser_3302_1_epsi, riser_3302_2_epsi)
rparams_series_tuple = (train_R300HD_series,train_riser_LAO_RHS_Zeta3D_series,train_riser_LAO_LHS_Zeta3D_series,train_riser_CAO_RHS_Zeta3D_series,train_riser_CAO_LHS_Zeta3D_series,train_riser_3302_2_3305_Zeta3D_series,train_riser_3302_1_Zeta3D_series,train_riser_3302_2_Zeta3D_series)

In [23]:
print('MTR Epsi:',MTR_epsi, '\triser_LAO_RHS_epsi:',riser_LAO_RHS_epsi, '\triser_LAO_LHS_epsi:',riser_LAO_LHS_epsi, '\triser_CAO_RHS_epsi:',riser_CAO_RHS_epsi, '\triser_CAO_LHS_epsi:',riser_CAO_LHS_epsi, '\triser_3302_1_epsi:',riser_3302_1_epsi, '\triser_3302_2_epsi:',riser_3302_2_epsi, '\triser_3302_2_3305_epsi:',riser_3302_2_3305_epsi)

MTR Epsi: 0.08120000000000001 	riser_LAO_RHS_epsi: 0.1 	riser_LAO_LHS_epsi: 0.1 	riser_CAO_RHS_epsi: 0.1 	riser_CAO_LHS_epsi: 0.1 	riser_3302_1_epsi: 0.1 	riser_3302_2_epsi: 0.1 	riser_3302_2_3305_epsi: 0.1


#### Train data points sequentially - Single Data Point Training Mode

In [24]:
%%time
network = PINN(epsi_tuple, activation_function_choice = 'linear', error_loss_choice = 'simplified') #cf_only, full, simplified, hyperbola_regularization                                                                                         
MSE_flowloss_hist_train, error_loss_hist_train, cf_hist_train, losses_hist = network.train_1p_seq_LS(learning_rates, network_dim, input_features, train_data[input_features], rparams_series_tuple, train_data_idx, restrictor_duct_areas, target_df, fddn_input_col_names, V_max_org, epochs = 25) 

Activation Function selected: linear
Loss Function & Delta-k Used: simplified

BEGIN Neural Network training for ['D2']
SEED value: 13
Nr of Hidden layers: 1 ;	Nr_Neurons in last hidden layer: 9
FIRST - INITIAL WEIGHTS & BIASES:
W1: [array([-0.712, 0.754, -0.0445, 0.452, 1.35, 0.532, 1.35, 0.861, 1.48])] <class 'numpy.ndarray'> Shape: (1, 9)
b1: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] <class 'numpy.ndarray'> Shape: (9,)
W2: [array([-0.37, -0.279, -0.446, 0.199, -0.086, 0.323, 0.112, 0.045]), array([0.76, 0.214, -0.00947, -0.348, 0.421, 0.337, -0.384, -0.0513]), array([0.0841, -0.58, -0.0983, 0.495, -0.571, 0.174, 0.669, -0.22]), array([-0.16, 0.0769, 0.182, 0.14, -0.535, -0.27, 0.0358, -0.112]), array([0.402, -0.114, -0.131, 0.654, -0.0289, -0.347, -0.000845, -0.0822]), array([0.251, 0.16, -0.118, -0.136, -0.187, 0.477, 0.712, -0.0203]), array([0.2, 0.127, -0.181, -0.185, -0.17, -0.68, -0.268, 0.16]), array([0.326, -0.0733, -0.133, 0.226, 0.726, -0.328, -0.274, -0.155]), array([0

Flowrate_delta (FDDN - LTR): [array([4.52, -0.296, 1.75, 1.52, 1.56])] <class 'numpy.ndarray'> (1, 5)
f1: -58.007415985744295 ;	lr1: 0.001 ;	MSE_FlowError: 5.664273030430415 ;	Error: 0.2896326180845549
f1 < 0 -> Skip f2 and use Brent

MTR-Cf: 0.796392756244714 ; riser_LAO_RHS-Cf: 0.983928786148039 ; riser_LAO_LHS-Cf: 1.0098433527497885 ; riser_CAO_RHS-Cf: 1.0077381180232983 ; riser_CAO_LHS-Cf: 0.9273107295145628 ; riser_3302_2_3305-Cf: 0.9677937352475432 ; riser_3302_1-Cf: 1.0091981142423165 ; riser_3302_2-Cf: 0.9148449524362366
Row ID: 12 	HoV: ['D2']
Flowrate_delta (FDDN - LTR): [array([1.31, -1.02, 1.51, 1.03, -0.212])] <class 'numpy.ndarray'> (1, 5)
zfunc, lr, f_lr: 0.0004530917798325778 1.5455143649348342

MTR-Cf: 0.796392756244714 ; riser_LAO_RHS-Cf: 0.983928786148039 ; riser_LAO_LHS-Cf: 1.0098433527497885 ; riser_CAO_RHS-Cf: 1.0077381180232983 ; riser_CAO_LHS-Cf: 0.9273107295145628 ; riser_3302_2_3305-Cf: 0.9677937352475432 ; riser_3302_1-Cf: 1.0091981142423165 ; riser_3302_2-Cf

Flowrate_delta (FDDN - LTR): [array([-0.772, -0.984, 0.767, -0.16, -0.395])] <class 'numpy.ndarray'> (1, 5)
f0: 9.162160172726725 ;	lr0: 0 ;	MSE_FlowError: 0.46687535796485724 ;	Error: 0.07887341379573383
Error is very LOW (< 0.1): 0.07887341379573383
Terminating Neural Network Training for D2
MSEFlow Loss History for D2: [20.233817906233405, 1.7998347263435974, 1.2285373326268147, 0.8121955566134498, 0.546280471155414, 0.46687535796485724]
Error Loss History for D2: [0.9537063674688369, 0.26625187645667997, 0.1874479129605826, 0.13612779129258124, 0.10426282938666757, 0.07887341379573383]
EARLY STOPPING ACTIVATED - Error Converged
Wall time: 3min 56s


In [25]:
cf_df = pd.DataFrame(data=cf_hist_train)
losses = pd.DataFrame(data=losses_hist)

In [26]:
mseflow_last_loss = []
error_last_loss = []
for i in range(len(losses)):
    loss1 = losses.MSE_flowloss[i][-1]
    loss2 = losses.Error_loss[i][-1]
    mseflow_last_loss.append(loss1)
    error_last_loss.append(loss2)

cf_df['Final_MSELoss'] = mseflow_last_loss
cf_df['Final_ErrorLoss'] = error_last_loss

In [27]:
cf_df['MaxEpochs'] = None # Initialize Empty Column for storing epochs trained

for i in range(len(losses['HoV'])):
    nr_epochs_trained = len(losses['MSE_flowloss'][i])
    cf_df['MaxEpochs'].loc[i] = nr_epochs_trained
    
cf_df.to_csv('data_output/NN_8CF_Output.csv',index=False)

In [28]:
# cf_df.sort_values(by = [MTR,'HoV'], inplace = True)
# print('Max no. of epochs for:',cf_df['HoV'].iloc[[cf_df.MaxEpochs.idxmax()]].values)
# cf_df.iloc[[cf_df.MaxEpochs.idxmax()]]

### Plot Training Losses

In [29]:
from bokeh.palettes import Category20,Colorblind,Spectral,Set1,Set2,YlGnBu,RdPu
temp_list = []
bokeh_palettes = [Colorblind,Set1,YlGnBu,RdPu,Set2]
for palette in bokeh_palettes:
    for key in palette.keys():
        temp_list.append(palette[key])
    
color_palette =  [y for x in temp_list for y in x] # ['red','blue']
num_lines = len(losses.HoV) # no. of lines to draw
colors = color_palette[0:num_lines]
labels = losses.HoV.values.tolist()

In [30]:
hover = HoverTool(names = ['EpochPoints'],tooltips = [('Epoch', '@x'), ('Error','@y')])
hover.point_policy='snap_to_data'

**MSE FlowError Loss**

In [31]:
p1 = figure(title = 'MSE FlowLoss for TZ3_8CF TRAIN data points', width=1250)
for i in range(num_lines):
    x = list(range(1,len(losses['MSE_flowloss'][i])+1))
    y1 = losses['MSE_flowloss'][i]    
    p1.line(x, y1, line_width=2, color=colors[i], alpha=0.8, name = 'EpochPoints',legend='MSE Flow Loss for {}'.format(labels[i]))    
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
p1.legend.click_policy="hide"
p1.add_tools(hover)
show(p1)

**Error Loss**

In [32]:
p2 = figure(title = 'Error (Loss) Function for TZ3_8CF TRAIN data', width=1250)
for i in range(num_lines):
    x = list(range(1,len(losses['Error_loss'][i])+1))     
    y2 = losses['Error_loss'][i]    
    p2.line(x, y2, line_width=2, color= colors[i], alpha=0.5, name = 'EpochPoints', legend='Error Loss for {}'.format(labels[i]))
    p2.circle(x, y2, line_width=2, color= colors[i], alpha=0.75, size = 8, name = 'EpochPoints', legend='Error Loss for {}'.format(labels[i]))
p2.yaxis.axis_label = "Error Loss"
p2.xaxis.axis_label = "Epochs"
p2.legend.click_policy="hide"
p2.xaxis.axis_label_text_font_size = '12pt'
p2.yaxis.axis_label_text_font_size = '12pt'
p2.xaxis.major_label_text_font_size= '11pt'
p2.xaxis.major_label_standoff = 15
p2.yaxis.major_label_text_font_size= '11pt'
p2.legend.label_text_font_size = '12pt'
p2.title.text_font_size = '12pt'
p2.add_tools(hover)
p2.output_backend = 'svg'
# export_svgs(p2, filename="plots/TZ3_8CF_ErrorLoss.svg")
show(p2)